In [1]:
## import libraries
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
!pip install wget
import wget

import sys, os, re, gc
from random import sample
import numpy.linalg as LA
import datetime
from sklearn.neighbors import NearestCentroid, NearestNeighbors

from sklearn.preprocessing import normalize, Normalizer
from sklearn.decomposition import KernelPCA, PCA
from imblearn.over_sampling import ADASYN, SMOTE, SVMSMOTE, KMeansSMOTE, BorderlineSMOTE
from sklearn.manifold import TSNE

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=19ee6c75ba41eff5134b14cbe2b1ec035360b1970d19c097d288091f3e29bea1
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
acp_feature_extraction = wget.download('https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/utils/acp_feature_extraction.py')
ACP_DL740_path = wget.download('https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/trained_models/ACP_DL_740_3.keras')
ACP_MHCNN740_path = wget.download('https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/trained_models/ACP_MHCNN.h5')
ACP_LSE740_path = wget.download('https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/trained_models/ACP-LSE/ACP-LSE2.h5')

ACP_DL240_path = wget.download('https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/trained_models/ACP_DL_240_0.keras')
ACP_MHCNN240_path = wget.download('https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/trained_models/model-240.h5')
ACP_LSE240_path = wget.download('https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/trained_models/ACP-LSE_240_1.keras')

pred_ACP_240 = 'https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/data/pred_seq_240_ACP.txt'
pred_ACP_740 = 'https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/data/pred_seq_740_ACP.txt'
pred_nonACP_240 = 'https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/data/pred_seq_240_nonACP.txt'
pred_nonACP_740 = 'https://raw.githubusercontent.com/mhdshl/LSE-Seq-synthesis/main/data/pred_seq_740_nonACP.txt'

from acp_feature_extraction import *

In [3]:
ACP_DL_240 = tf.keras.models.load_model(ACP_DL240_path)
ACP_MHCNN_240 = tf.keras.models.load_model(ACP_MHCNN240_path)
ACP_LSE_240 = tf.keras.models.load_model(ACP_LSE240_path)

ACP_DL_740 = tf.keras.models.load_model(ACP_DL740_path)
ACP_MHCNN_740 = tf.keras.models.load_model(ACP_MHCNN740_path)
ACP_LSE_740 = tf.keras.models.load_model(ACP_LSE740_path)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


## ACP-MHCNN



### ACP 240

In [4]:
BPF240_nonACP = numeric(dBPF, pred_nonACP_240)
dBIT240_nonACP = numeric(dBIT, pred_nonACP_240)
dBLOSUM240_nonACP = numeric(dBLOSUM, pred_nonACP_240)

print(BPF240_nonACP.shape)
print(dBIT240_nonACP.shape)
print(dBLOSUM240_nonACP.shape)

y_pred_240nonACP_MHCNN = ACP_MHCNN_240.predict([BPF240_nonACP, dBIT240_nonACP, dBLOSUM240_nonACP])
print(y_pred_240nonACP_MHCNN.shape)
pred_nonACP240_MHCNN = np.argmax(y_pred_240nonACP_MHCNN, axis=1)
print('Synthesized nonACP240 Positive',pred_nonACP240_MHCNN.shape)

BPF240_ACP = numeric(dBPF, pred_ACP_240)
dBIT240_ACP = numeric(dBIT, pred_ACP_240)
dBLOSUM240_ACP = numeric(dBLOSUM, pred_ACP_240)

print(BPF240_ACP.shape)
print(dBIT240_ACP.shape)
print(dBLOSUM240_ACP.shape)

y_pred_240ACP_MHCNN = ACP_MHCNN_240.predict([BPF240_ACP, dBIT240_ACP, dBLOSUM240_ACP])
print(y_pred_240ACP_MHCNN.shape)
pred_ACP240_MHCNN = np.argmax(y_pred_240ACP_MHCNN, axis=1)
print('Synthesized ACP240 Positive', pred_ACP240_MHCNN.shape)

130021/130021 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(4000, 25, 20)
(4000, 25, 31)
(4000, 25, 20)
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step
(4000, 2)
Synthesized nonACP240 Positive (4000,)
139547/139547 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(4000, 25, 20)
(4000, 25, 31)
(4000, 25, 20)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
(4000, 2)
Synthesized ACP240 Positive (4000,)


### ACP740

In [5]:
BPF740_nonACP = numeric(dBPF, pred_nonACP_740)
dBIT740_nonACP = numeric(dBIT, pred_nonACP_740)
dBLOSUM740_nonACP = numeric(dBLOSUM, pred_nonACP_740)

print(BPF740_nonACP.shape)
print(dBIT740_nonACP.shape)
print(dBLOSUM740_nonACP.shape)

y_pred_740nonACP_MHCNN = ACP_MHCNN_740.predict([BPF740_nonACP, dBIT740_nonACP, dBLOSUM740_nonACP])
print(y_pred_740nonACP_MHCNN.shape)
pred_nonACP740_MHCNN = np.argmax(y_pred_740nonACP_MHCNN, axis=1)
print('Synthesized nonACP740 Positive',pred_nonACP740_MHCNN.shape)

BPF740_ACP = numeric(dBPF, pred_ACP_740)
dBIT740_ACP = numeric(dBIT, pred_ACP_740)
dBLOSUM740_ACP = numeric(dBLOSUM, pred_ACP_740)

print(BPF740_ACP.shape)
print(dBIT740_ACP.shape)
print(dBLOSUM740_ACP.shape)

y_pred_740ACP_MHCNN = ACP_MHCNN_740.predict([BPF740_ACP, dBIT740_ACP, dBLOSUM740_ACP])
print(y_pred_740ACP_MHCNN.shape)
pred_ACP740_MHCNN = np.argmax(y_pred_740ACP_MHCNN, axis=1)
print('Synthesized ACP740 Positive', pred_ACP740_MHCNN.shape)

153142/153142 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(4000, 25, 20)
(4000, 25, 31)
(4000, 25, 20)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
(4000, 2)
Synthesized nonACP740 Positive (4000,)
129180/129180 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(4000, 25, 20)
(4000, 25, 31)
(4000, 25, 20)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
(4000, 2)
Synthesized ACP740 Positive (4000,)


## ACP-DL

### ACP240

In [6]:
data_dim = 483
timesteps = 1

bpf240_nonACP, kmer240_nonACP, label240_nonACP = prepare_feature_ACP_DL(pred_nonACP_240)
print(bpf240_nonACP.shape)
print(kmer240_nonACP.shape)
print(len(label240_nonACP))

x240_nonACP = np.concatenate((bpf240_nonACP, kmer240_nonACP), axis=1)
x240_nonACP = np.reshape(x240_nonACP, (len(x240_nonACP), timesteps, data_dim))
print(x240_nonACP.shape)

y_240_nonACP_DL = ACP_DL_240.predict(x240_nonACP)
print(y_240_nonACP_DL.shape)

pred_240_nonACP_DL = np.round(y_240_nonACP_DL)
pred_240_nonACP_DL = to_categorical(pred_240_nonACP_DL)
pred_240_nonACP_DL = np.argmax(pred_240_nonACP_DL, axis=1)
pred_240_nonACP_DL = pred_240_nonACP_DL.astype(int)
print('ACP-DL 240 nonACP',pred_240_nonACP_DL.shape)

bpf240_ACP, kmer240_ACP, label240_ACP = prepare_feature_ACP_DL(pred_ACP_240)
print(bpf240_ACP.shape)
print(kmer240_ACP.shape)
print(len(label240_ACP))

x240_ACP = np.concatenate((bpf240_ACP, kmer240_ACP), axis=1)
x240_ACP = np.reshape(x240_ACP, (len(x240_ACP), timesteps, data_dim))
print(x240_ACP.shape)

y_240_ACP_DL = ACP_DL_240.predict(x240_nonACP)
print(y_240_ACP_DL.shape)

pred_240_ACP_DL = np.round(y_240_ACP_DL)
pred_240_ACP_DL = to_categorical(pred_240_ACP_DL)
pred_240_ACP_DL = np.argmax(pred_240_ACP_DL, axis=1)
pred_240_ACP_DL = pred_240_ACP_DL.astype(int)
print(pred_240_ACP_DL.shape)

(4000, 140)
(4000, 343)
4000
(4000, 1, 483)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
(4000, 1)
ACP-DL 240 nonACP (4000,)
(4000, 140)
(4000, 343)
4000
(4000, 1, 483)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
(4000, 1)
(4000,)


### ACP740

In [7]:
data_dim = 483
timesteps = 1

bpf740_nonACP, kmer740_nonACP, label740_nonACP = prepare_feature_ACP_DL(pred_nonACP_740)
print(bpf740_nonACP.shape)
print(kmer740_nonACP.shape)
print(len(label740_nonACP))

x740_nonACP = np.concatenate((bpf740_nonACP, kmer740_nonACP), axis=1)
x740_nonACP = np.reshape(x740_nonACP, (len(x740_nonACP), timesteps, data_dim))
print(x740_nonACP.shape)

y_740_nonACP_DL = ACP_DL_740.predict(x240_nonACP)
print(y_740_nonACP_DL.shape)

pred_740_nonACP_DL = np.round(y_740_nonACP_DL)
pred_740_nonACP_DL = to_categorical(pred_740_nonACP_DL)
pred_740_nonACP_DL = np.argmax(pred_740_nonACP_DL, axis=1)
pred_740_nonACP_DL = pred_740_nonACP_DL.astype(int)
print('ACP-DL 740 nonACP',pred_740_nonACP_DL.shape)

bpf740_ACP, kmer740_ACP, label740_ACP = prepare_feature_ACP_DL(pred_ACP_740)
print(bpf740_ACP.shape)
print(kmer740_ACP.shape)
print(len(label740_ACP))

x740_ACP = np.concatenate((bpf740_ACP, kmer740_ACP), axis=1)
x740_ACP = np.reshape(x740_ACP, (len(x740_ACP), timesteps, data_dim))
print(x740_ACP.shape)

y_740_ACP_DL = ACP_DL_740.predict(x740_nonACP)
print(y_740_ACP_DL.shape)

pred_740_ACP_DL = np.round(y_740_ACP_DL)
pred_740_ACP_DL = to_categorical(pred_740_ACP_DL)
pred_740_ACP_DL = np.argmax(pred_740_ACP_DL, axis=1)
pred_740_ACP_DL = pred_740_ACP_DL.astype(int)
print(pred_740_ACP_DL.shape)

(4000, 140)
(4000, 343)
4000
(4000, 1, 483)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
(4000, 1)
ACP-DL 740 nonACP (4000,)
(4000, 140)
(4000, 343)
4000
(4000, 1, 483)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
(4000, 1)
(4000,)


## ACP-LSE

### ACP240

In [8]:
[Data240_nonACP, Label240_nonACP] = Convert_Seq2CKSAAP(prepare_feature_for_CKSAAP(pred_nonACP_240), gap=8)
print(Data240_nonACP.shape)
print(Label240_nonACP.shape)

pred_240_nonACP, _ = ACP_LSE_240.predict(Data240_nonACP)
print(pred_240_nonACP.shape)

pred_240_nonACP_LSE = np.argmax(pred_240_nonACP, axis=1)
print('nonACP240 LSE Pred',pred_240_nonACP_LSE.shape)

[Data240_ACP, Label240_ACP] = Convert_Seq2CKSAAP(prepare_feature_for_CKSAAP(pred_ACP_240), gap=8)
print(Data240_ACP.shape)
print(Label240_ACP.shape)

pred_240_ACP, _ = ACP_LSE_240.predict(Data240_ACP)
print(pred_240_ACP.shape)

pred_240_ACP_LSE = np.argmax(pred_240_ACP, axis=1)
print('ACP240 LSE Pred',pred_240_ACP_LSE.shape)

num pos: 4000 num neg: 0
(4000, 3600)
(4000, 1)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
(4000, 2)
nonACP240 LSE Pred (4000,)
num pos: 4000 num neg: 0
(4000, 3600)
(4000, 1)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
(4000, 2)
ACP240 LSE Pred (4000,)


### ACP740

In [9]:
[Data740_nonACP, Label740_nonACP] = Convert_Seq2CKSAAP(prepare_feature_for_CKSAAP(pred_nonACP_740), gap=8)
print(Data740_nonACP.shape)
print(Label740_nonACP.shape)

pred_740_nonACP, _ = ACP_LSE_740.predict(Data740_nonACP)
print(pred_740_nonACP.shape)

pred_740_nonACP_LSE = np.argmax(pred_740_nonACP, axis=1)
print('nonACP740 LSE Pred',pred_740_nonACP_LSE.shape)

[Data740_ACP, Label740_ACP] = Convert_Seq2CKSAAP(prepare_feature_for_CKSAAP(pred_ACP_740), gap=8)
print(Data740_ACP.shape)
print(Label740_ACP.shape)

pred_740_ACP, _ = ACP_LSE_740.predict(Data740_ACP)
print(pred_740_ACP.shape)

pred_740_ACP_LSE = np.argmax(pred_740_ACP, axis=1)
print('ACP740 LSE Pred',pred_740_ACP_LSE.shape)

num pos: 4000 num neg: 0
(4000, 3600)
(4000, 1)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
(4000, 2)
nonACP740 LSE Pred (4000,)
num pos: 4000 num neg: 0
(4000, 3600)
(4000, 1)
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
(4000, 2)
ACP740 LSE Pred (4000,)


In [10]:
matching_240_nonACP_ind = []
for i in range(0, Label240_nonACP.shape[0]):
  if pred_nonACP240_MHCNN[i] == pred_240_nonACP_DL[i] == 0:# pred_240_nonACP_LSE[i] == 0:
    matching_240_nonACP_ind.append(i)
print('NonACP sequences classified as non-ACPs for ACP240 dataset')
print(70*'-')
print('Number of non-ACP (ACP240) Classified as non-ACP (rank=2):',len(matching_240_nonACP_ind))
print('Total number of sequences (trained):',Label240_nonACP.shape[0])
print('Accuracy = ', len(matching_240_nonACP_ind)/Label240_nonACP.shape[0])

NonACP sequences classified as non-ACPs for ACP240 dataset
----------------------------------------------------------------------
Number of non-ACP (ACP240) Classified as non-ACP (rank=2): 1445
Total number of sequences (trained): 4000
Accuracy =  0.36125


In [11]:
matching_240_ACP_ind = []
for i in range(0, Label240_ACP.shape[0]):
  if pred_ACP240_MHCNN[i] == pred_240_ACP_DL[i] == 1:#pred_240_ACP_LSE[i] == 1:
    matching_240_ACP_ind.append(i)
print('ACP sequences classified as ACPs for ACP240 dataset')
print(70*'-')
print('Number of Positive ACPs (rank = 2) for ACPs generated using our model:',len(matching_240_ACP_ind))
print('Total number of sequences (trained):',Label240_ACP.shape[0])
print('Accuracy = ', len(matching_240_ACP_ind)/Label240_ACP.shape[0])

ACP sequences classified as ACPs for ACP240 dataset
----------------------------------------------------------------------
Number of Positive ACPs (rank = 2) for ACPs generated using our model: 1967
Total number of sequences (trained): 4000
Accuracy =  0.49175


In [12]:
matching_740_nonACP_ind = []
for i in range(0, Label740_nonACP.shape[0]):
  if pred_nonACP740_MHCNN[i] == pred_740_nonACP_DL[i] == 0:#pred_740_nonACP_LSE[i] == 1:
    matching_740_nonACP_ind.append(i)
print('NonACP sequences classified as non-ACPs for ACP740 dataset')
print(70*'-')
print('Number of Positive ACP740 (rank = 2) for ACPs generated using our model:',len(matching_740_nonACP_ind))
print('Total number of sequences (trained):',Label740_nonACP.shape[0])
print('Accuracy = ', len(matching_740_nonACP_ind)/Label740_nonACP.shape[0])

NonACP sequences classified as non-ACPs for ACP740 dataset
----------------------------------------------------------------------
Number of Positive ACP740 (rank = 2) for ACPs generated using our model: 3231
Total number of sequences (trained): 4000
Accuracy =  0.80775


In [13]:
matching_740_ACP_ind = []
for i in range(0, Label740_ACP.shape[0]):
  if pred_ACP740_MHCNN[i] == pred_740_ACP_DL[i] == 1:#pred_740_ACP_LSE[i] == 1:
    matching_740_ACP_ind.append(i)
print('ACP sequences classified as ACPs for ACP740 dataset')
print(70*'-')
print('Number of Positive ACP740 (rank = 2) for ACPs generated using our model:',len(matching_740_ACP_ind))
print('Total number of sequences (trained):',Label740_ACP.shape[0])
print('Accuracy = ', len(matching_740_ACP_ind)/Label740_ACP.shape[0])

ACP sequences classified as ACPs for ACP740 dataset
----------------------------------------------------------------------
Number of Positive ACP740 (rank = 2) for ACPs generated using our model: 1909
Total number of sequences (trained): 4000
Accuracy =  0.47725


### Filtered positive and negative sequence extraction and saving as txt files

In [14]:
def seq_save(gen_seq_path, match_seq_ind, seq_title = 'ACP', file_name = 'ACP240_filter.txt', label = '0'):
  seq_file = wget.download(gen_seq_path)
  new_list=[]
  # label = []
  seq_list = []
  filter_seq = []
  lis = []
  with open(seq_file, 'r') as fp:
    for line in fp:
        if line[0] == '>':
          values = line[1:].strip().split('|')
          label_temp = values[1]
          proteinName = values[0]
          proteinName_1=proteinName.split("_")
          new_list.append(proteinName_1[0])

        else:
            seq = line[:-1]
            seq_list.append(seq)
    f240 = open(file_name, 'w')
    for i in match_seq_ind:
        f240.write('>'+seq_title+'_'+str(i)+'|'+label+'\n')
        f240.write(seq_list[i]+'\n')
        filter_seq.append(seq_list[i])
    f240.close()
    return filter_seq

In [15]:
nACP240_filter = seq_save(pred_nonACP_240, matching_240_nonACP_ind, 'nonACP', file_name = 'nonACP240_filter.txt', label = '4')
ACP240_filter = seq_save(pred_ACP_240, matching_240_ACP_ind, 'ACP', file_name = 'ACP240_filter.txt', label = '5')
nACP740_filter = seq_save(pred_nonACP_740, matching_740_nonACP_ind, 'nonACP', file_name = 'nonACP740_filter.txt', label = '6')
ACP740_filter = seq_save(pred_ACP_740, matching_740_ACP_ind, 'ACP', file_name = 'ACP740_filter.txt', label = '7')